In [ ]:
!pip install mplfinance


In [ ]:
import requests
import pandas as pd
import mplfinance as mpf

# 🔐 Access Token (Demo - Replace with your actual one if needed)
access_token = "YOUR_ACCESS_TOKEN_HERE"


In [ ]:
url = "https://api.topstepx.com/api/History/retrieveBars"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "accept": "application/json"
}
payload = {
    "contractId": "CON.F.US.MNQ.U25",
    "live": False,
    "startTime": "2025-06-24T00:00:00Z",
    "endTime": "2025-06-25T00:00:00Z",
    "unit": 2,
    "unitNumber": 5,
    "limit": 100,
    "includePartialBar": False
}
response = requests.post(url, json=payload, headers=headers)
bars = response.json()["bars"]
df = pd.DataFrame(bars)
df.rename(columns={"t": "Date", "o": "Open", "h": "High", "l": "Low", "c": "Close", "v": "Volume"}, inplace=True)
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", inplace=True)
df.tail()


In [ ]:
ha_df = df.copy()
ha_df['HA_Close'] = (ha_df['Open'] + ha_df['High'] + ha_df['Low'] + ha_df['Close']) / 4
ha_open = [(ha_df['Open'][0] + ha_df['Close'][0]) / 2]
for i in range(1, len(ha_df)):
    ha_open.append((ha_open[i-1] + ha_df['HA_Close'][i-1]) / 2)
ha_df['HA_Open'] = ha_open
ha_df['HA_High'] = ha_df[['High', 'HA_Open', 'HA_Close']].max(axis=1)
ha_df['HA_Low'] = ha_df[['Low', 'HA_Open', 'HA_Close']].min(axis=1)


In [ ]:
# === MA Calculation
ema1 = ha_df['Close'].ewm(span=9, adjust=False).mean()
ema2 = ha_df['Close'].ewm(span=29, adjust=False).mean()
ema_trend = ha_df['Close'].ewm(span=100, adjust=False).mean()

# === Conditions
ha_df['long_signal'] = (ema1 > ema2) & (ha_df['HA_Close'] > ha_df['HA_Open']) & (ha_df['HA_High'] > ema_trend)
ha_df['short_signal'] = (ema1 < ema2) & (ha_df['HA_Close'] < ha_df['HA_Open']) & (ha_df['HA_Low'] < ema_trend)

# === Last Signals
last_long = ha_df[ha_df['long_signal']].iloc[-1:] if ha_df['long_signal'].any() else None
last_short = ha_df[ha_df['short_signal']].iloc[-1:] if ha_df['short_signal'].any() else None

if last_long is not None:
    print("✅ آخرین سیگنال خرید:")
    print(last_long[['Open', 'High', 'Low', 'Close']])
if last_short is not None:
    print("✅ آخرین سیگنال فروش:")
    print(last_short[['Open', 'High', 'Low', 'Close']])
if last_long is None and last_short is None:
    print("❌ سیگنال فعالی یافت نشد.")


In [ ]:
mpf.plot(df, type='candle', style='charles', title='MNQ - 5m Chart', ylabel='Price', volume=True)
